In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

from os import listdir
train_data_li = listdir('./train/')
train_data_li

import sys
mod = sys.modules[__name__]
for name in train_data_li:
    setattr(mod, name.split('.')[0], pd.read_csv('./train/' + name))
    
print(train_activity.shape, train_combat.shape, 
      train_pledge.shape, train_trade.shape, 
      train_payment.shape, train_label.shape)

(1601834, 17) (1574671, 13) (1083384, 15) (1849325, 11) (66432, 3) (40000, 3)


- total spent label 생성

In [2]:
train_label['total_spent'] = train_label['survival_time'] * train_label['amount_spent']

In [3]:
total_spent = train_label.set_index('acc_id')['total_spent']

### 소비 관련 feature 생성
- Assumption
    1. 28일 간 소비가 컸던 사람은 계속하여 과금을 할 것이다.
    2. 거래를 자주 하는 캐릭터는 과금을 할 가능성이 높을 것이다.
    3. 공성전이 활발한 혈맹에 들어있는 유저는 과금을 할 가능성이 높을 것이다.
    4. 게임을 새로 시작하고 의욕에 불타는 유저(고인물이 될 가능성이 있는 유저)는 과금을 추가로 할 가능성이 높을 것이다.
    5. 캐릭터가 약하다고 느낀 플레이어는 과금으로 캐릭터를 강화하려고 할 가능성이 클 것이다.
    6. 장사꾼 캐릭터는 과금을 많이 할 것인가
    7. pvp를 자주하는 유저의 과금량은 어떻게 될 것인가

In [4]:
#1 payment 기록이 있는 유저의 28일간 총 과금량
ts28_acc = train_payment.groupby(['acc_id'])['amount_spent'].sum()

In [5]:
train_trade['hour'] = train_trade['time'].map(lambda x : x.split(':')[0])
train_trade['minute'] = train_trade['time'].map(lambda x : x.split(':')[1])
train_trade['second'] = train_trade['time'].map(lambda x : x.split(':')[2])

In [6]:
train_trade = train_trade.sort_values(['day', 'hour', 'minute', 'second'])
train_trade.head()

,day,time,type,server,source_acc_id,source_char_id,target_acc_id,target_char_id,item_type,item_amount,item_price,hour,minute,second
312052,1,00:00:00,0,as,104177,163760,50138,388244,etc,8.868841e-07,2.994625,00,00,00
49265,1,00:00:01,1,ag,22181,330660,97774,378791,adena,1.893617e+00,NaN,00,00,01
97977,1,00:00:01,1,ag,97774,378791,22181,330660,etc,1.143361e-05,NaN,00,00,01
127986,1,00:00:01,1,aq,74334,355483,87174,389623,adena,1.198492e-02,NaN,00,00,01
157271,1,00:00:01,1,ag,97774,378791,22181,330660,accessory,2.396984e-08,NaN,00,00,01


In [7]:
train_trade = train_trade.drop('time', axis=1)

In [8]:
class_dict = {0 : '군주',
              1 : '기사',
              2 : '요정',
              3 : '마법사',
              4 : '다크엘프',
              5 : '용기사',
              6 : '환술사',
              7 : '전사'}

In [9]:
# 직업
acc_char_class = train_combat.groupby(['acc_id', 'char_id'])['class'].mean() # 클래스 변동이 없을 경우

In [10]:
acc_char_class = acc_char_class.map(lambda x : class_dict[x])

In [11]:
acc_char_class.head()

acc_id  char_id
2       233598      기사
5       22260       요정
        175813      요정
        321231      요정
8       43437      용기사
Name: class, dtype: object

In [14]:
# playtime
acc_char_playtime = train_activity.groupby(['acc_id', 'char_id'])['playtime'].sum()

- pledge feature selection

In [ ]:
# acc_char_pledge = train_pledge.groupby(['acc_id', 'char_id'])['pledge_id'].count()

# train_pledge[train_pledge['pledge_id'] == 39601]
# train_pledge[train_pledge['pledge_id'] == 35333]

In [15]:
use_col = ['pledge_id', 'day', 'acc_id', 'char_id', 'server']
test2 = train_pledge[use_col].sort_values(use_col)

main_server_ind, bat_server_ind = [], []
prepare_ind = test2.groupby(['pledge_id', 'day', 'acc_id', 'char_id']).count().values.ravel()
for i in prepare_ind:
    if i > 1:
        for j in range(i):
            main_server_ind.append(False)
            bat_server_ind.append(True)
    else:
        main_server_ind.append(True)
        bat_server_ind.append(False)

In [16]:
train_main_pledge = train_pledge.sort_values(use_col)[main_server_ind]
train_bat_pledge = train_pledge.sort_values(use_col)[bat_server_ind]

- 혈맹을 옮긴 플레이어 

In [56]:
pledge_id_stats = train_main_pledge.groupby(['acc_id', 'char_id', 'day']).agg({'pledge_id' : ['first', 'last', 'count']})

In [57]:
pledge_id_stats[('pledge_id', 'count')].value_counts()

1    1001887
2        329
Name: (pledge_id, count), dtype: int64

In [59]:
train_pledge[(train_pledge['acc_id'] == 350) & (train_pledge['char_id'] == 92896)]

,day,acc_id,char_id,server,pledge_id,play_char_cnt,combat_char_cnt,pledge_combat_cnt,random_attacker_cnt,random_defender_cnt,same_pledge_cnt,temp_cnt,etc_cnt,combat_play_time,non_combat_play_time
1070,1,350,92896,aa,8419,2.021002,0.866144,0.036425,6.777991,0.429188,0.000000,1.681012,0.233583,2.365386,0.0
37050,2,350,92896,aa,8419,1.624019,0.685697,0.072851,2.356159,0.000000,0.000000,0.494415,0.488400,2.298899,0.0
73905,3,350,92896,aa,8419,1.551841,0.505250,0.000000,4.421833,0.143063,0.000000,0.692181,0.438852,2.258937,0.0
111081,4,350,92896,aa,8419,1.876645,1.010501,0.020034,1.387874,0.858376,1.591434,0.395532,0.629966,2.605735,0.0
147990,5,350,92896,aa,8419,1.984913,1.190948,0.127489,0.000000,0.429188,0.000000,0.791064,2.073932,3.235277,0.0
184302,6,350,92896,aa,8419,1.948823,1.046590,0.047353,2.840301,0.572251,0.000000,0.988830,1.196227,2.618592,0.0
220591,7,350,92896,aa,8419,1.876645,1.154858,0.120204,0.580971,0.572251,0.795717,0.988830,1.988993,3.314273,0.0
258106,8,350,92896,aa,8419,1.948823,1.118769,0.122025,2.452987,0.429188,0.000000,0.494415,1.288244,2.410560,0.0
295728,9,350,92896,aa,8419,1.624019,0.902233,0.131131,2.840301,0.000000,2.785010,0.791064,1.210383,2.277702,0.0
333721,10,350,92896,aa,8419,1.479662,0.721786,0.030962,0.000000,0.071531,0.000000,0.395532,0.785687,1.932178,0.0


In [58]:
pledge_id_stats[(pledge_id_stats[('pledge_id', 'count')] > 1).values]

pledge_id             
                       first   last count
acc_id char_id day                       
350    92896   19       8419  10843     2
1284   160706  16        934   1442     2
               26        934   1442     2
2740   225674  4       15459  35573     2
               19      15459  35573     2
3946   316927  5       10641  32985     2
               12      10641  32985     2
               26      10641  32985     2
5135   176634  5       32985  34086     2
               10      32985  34086     2
               11      32985  34086     2
               16      32985  34086     2
       313468  5       32026  34086     2
6335   369156  26      21479  25030     2
6507   140271  19      20342  21739     2
8946   75716   20      18009  21739     2
9262   363060  23      15459  23745     2
9458   177322  21       1188   4937     2
11217  353331  26      30520  38975     2
11277  14077   15      15883  20876     2
11734  361244  17       4459  20876     2
12031  196195  21      19949  24131     2
12076  336279  20      18009  21739     2
12709  245821  5        2008  36949     2
18583  264148  5       15459  23950     2
               12      15459  23950     2
               19      15459  23950     2
               26      15459  23950     2
18601  69959   8       30833  35254     2
18800  105323  3       15459  23745     2
...                      ...    ...   ...
117231 229545  25      11897  25706     2
119015 77818   20      11897  25706     2
120434 197434  12      17869  30469     2
121822 177392  27       5427  36997     2
122090 124353  5       15459  23745     2
               19      15459  23745     2
               26      15459  23745     2
122651 88366   26        368  29152     2
124641 95592   20      18009  21739     2
125708 12042   11       7904  14353     2
126699 180985  5       12653  32985     2
               12      12653  32985     2
               19      12653  32985     2
               23      12653  32985     2
               25      12653  32985     2
126768 196986  26      24319  31076     2
127609 154083  12      29269  39000     2
128610 211397  8        1000  24801     2
129337 49384   20      18009  21739     2
129809 358738  24      10823  33648     2
129905 353518  5        8419  10843     2
               12       8419  10843     2
130148 272577  5        8419  10843     2
               12       8419  10843     2
               17       8419  10843     2
               19       8419  10843     2
               26       8419  10843     2
130164 370682  26      23128  29152     2
130218 162756  26      21479  25030     2
130244 349781  15      14362  39601     2

[329 rows x 3 columns]

In [23]:
train_main_pledge.groupby(['pledge_id', 'day', 'acc_id', 'char_id']).mean()

play_char_cnt  combat_char_cnt  \
pledge_id day acc_id char_id                                   
2         1   39448  277729        0.036089         0.000000   
          23  39448  277729        0.036089         0.000000   
3         1   538    300207        0.793965         0.505250   
              15434  30377         0.793965         0.505250   
              25983  342356        0.793965         0.505250   
              26564  287697        0.793965         0.505250   
              61420  125555        0.793965         0.505250   
              65886  295854        0.793965         0.505250   
              80475  224509        0.793965         0.505250   
              86844  180762        0.793965         0.505250   
              90220  300646        0.793965         0.505250   
              92806  10541         0.793965         0.505250   
              97401  79739         0.793965         0.505250   
              99765  337220        0.793965         0.505250   
              102371 378273        0.793965         0.505250   
          2   538    300207        0.793965         0.469161   
              15434  30377         0.793965         0.469161   
              25983  342356        0.793965         0.469161   
              61420  125555        0.793965         0.469161   
              65886  295854        0.793965         0.469161   
              80475  224509        0.793965         0.469161   
              86844  180762        0.793965         0.469161   
              90220  300646        0.793965         0.469161   
              92806  10541         0.793965         0.469161   
              97401  79739         0.793965         0.469161   
              99765  337220        0.793965         0.469161   
              102371 378273        0.793965         0.469161   
          3   538    300207        0.757876         0.577429   
              15434  30377         0.757876         0.577429   
              25983  342356        0.757876         0.577429   
...                                     ...              ...   
39623     27  50404  190632        0.541340         0.433072   
              56947  274359        0.541340         0.433072   
              58673  356941        0.541340         0.433072   
              60431  27977         0.541340         0.433072   
              65043  312630        0.541340         0.433072   
              75536  390073        0.541340         0.433072   
              97989  174730        0.541340         0.433072   
              110849 184418        0.541340         0.433072   
              111225 246186        0.541340         0.433072   
              115714 68441         0.541340         0.433072   
              125353 350156        0.541340         0.433072   
          28  4460   341982        0.505250         0.469161   
              31883  61692         0.505250         0.469161   
              40463  357131        0.505250         0.469161   
              50404  190632        0.505250         0.469161   
              56947  274359        0.505250         0.469161   
              58673  356941        0.505250         0.469161   
              60431  27977         0.505250         0.469161   
              65043  312630        0.505250         0.469161   
              75536  390073        0.505250         0.469161   
              97989  174730        0.505250         0.469161   
              110849 184418        0.505250         0.469161   
              111225 246186        0.505250         0.469161   
              115714 68441         0.505250         0.469161   
              125353 350156        0.505250         0.469161   
39624     6   39832  212101        0.072179         0.000000   
          19  39832  212101        0.108268         0.036089   
          22  39832  212101        0.072179         0.000000   
          23  39832  212101        0.072179         0.000000   
39625     28  70097  207006        0.036089         0.000000  

### Q) bd를 주섭으로 삼는 혈맹은 a_서버로 넘어가는 친구들이 존재하는지?

- masking data => 나눠주면 std 상쇄

---------------------------------------------------------------------
# 아래는 아직 보지마!

- trade 탐구

In [63]:
# 자기 캐릭으로 옮기는 지
train_trade[train_trade['source_acc_id'] == train_trade['target_acc_id']] # 없다.

,day,type,server,source_acc_id,source_char_id,target_acc_id,target_char_id,item_type,item_amount,item_price,hour,minute,second


In [68]:
acc_id = train_label.acc_id.values

In [178]:
trade_source_acc_id = train_trade[np.isin(train_trade['source_acc_id'].values, acc_id)]
trade_target_acc_id = train_trade[np.isin(train_trade['target_acc_id'].values, acc_id)]

In [179]:
trade_all_acc_id = train_trade[np.logical_and(
    np.isin(train_trade['source_acc_id'].values, acc_id),
    np.isin(train_trade['target_acc_id'].values, acc_id))]

In [180]:
source_gate = np.logical_and(
    np.isin(train_trade['source_acc_id'].values, acc_id),
    np.logical_xor(
        np.isin(train_trade['source_acc_id'].values, acc_id),
        np.isin(train_trade['target_acc_id'].values, acc_id)
    )
)

target_gate = np.logical_and(
    np.isin(train_trade['target_acc_id'].values, acc_id),
    np.logical_xor(
        np.isin(train_trade['target_acc_id'].values, acc_id),
        np.isin(train_trade['source_acc_id'].values, acc_id)
    )
)

In [181]:
trade_source_acc_id.shape[0], trade_target_acc_id.shape[0]

(1403720, 1280083)

In [182]:
trade_all_acc_id.shape[0] + source_gate.sum()

1403720

In [183]:
trade_all_acc_id.shape[0] + target_gate.sum()

1280083

In [184]:
trade_only_source_acc_id = train_trade[source_gate]
trade_only_target_acc_id = train_trade[target_gate]

In [185]:
trade_all_acc_id.shape[0], trade_source_acc_id.shape[0], trade_target_acc_id.shape[0], trade_only_source_acc_id.shape[0], trade_only_target_acc_id.shape[0]

(834478, 1403720, 1280083, 569242, 445605)

In [268]:
acc_char_source_t0 = trade_only_source_acc_id[trade_only_source_acc_id['type'] == 0].groupby(['source_acc_id', 'source_char_id'])['server'].count()
acc_char_source_t0.index.names = ['acc_id', 'char_id']
acc_char_source_t0.name = 'source2other_t0'

acc_char_source_t1 = trade_only_source_acc_id[trade_only_source_acc_id['type'] == 1].groupby(['source_acc_id', 'source_char_id'])['server'].count()
acc_char_source_t1.index.names = ['acc_id', 'char_id']
acc_char_source_t1.name = 'source2other_t1'

acc_char_target_t0 = trade_only_target_acc_id[trade_only_target_acc_id['type'] == 0].groupby(['target_acc_id', 'target_char_id'])['server'].count()
acc_char_target_t0.index.names = ['acc_id', 'char_id']
acc_char_target_t0.name = 'other2target_t0'

acc_char_target_t1 = trade_only_target_acc_id[trade_only_target_acc_id['type'] == 1].groupby(['target_acc_id', 'target_char_id'])['server'].count()
acc_char_target_t1.index.names = ['acc_id', 'char_id']
acc_char_target_t1.name = 'other2target_t1'

In [269]:
acc_char_source2_t0 = trade_all_acc_id[trade_all_acc_id['type'] == 0].groupby(['source_acc_id', 'source_char_id'])['server'].count()
acc_char_source2_t0.index.names = ['acc_id', 'char_id']
acc_char_source2_t0.name = 'source2label_t0'

acc_char_source2_t1 = trade_all_acc_id[trade_all_acc_id['type'] == 1].groupby(['source_acc_id', 'source_char_id'])['server'].count()
acc_char_source2_t1.index.names = ['acc_id', 'char_id']
acc_char_source2_t1.name = 'source2label_t1'

acc_char_target2_t0 = trade_all_acc_id[trade_all_acc_id['type'] == 0].groupby(['target_acc_id', 'target_char_id'])['server'].count()
acc_char_target2_t0.index.names = ['acc_id', 'char_id']
acc_char_target2_t0.name = 'label2target_t0'

acc_char_target2_t1 = trade_all_acc_id[trade_all_acc_id['type'] == 1].groupby(['target_acc_id', 'target_char_id'])['server'].count()
acc_char_target2_t1.index.names = ['acc_id', 'char_id']
acc_char_target2_t1.name = 'label2target_t1'

In [220]:
train_activity.groupby(['acc_id', 'char_id', 'day'])['server'].count().value_counts()

1    1518104
2      31329
3       7024
Name: server, dtype: int64

In [221]:
train_activity.groupby(['acc_id', 'char_id', 'day'])['server'].count()[train_activity.groupby(['acc_id', 'char_id', 'day'])['server'].count() > 1] # 서버 이동하면서 플레이하는 유저 존재

acc_id  char_id  day
20      149667   16     2
                 17     2
                 18     2
                 22     2
                 23     2
                 24     2
                 25     2
                 27     2
                 28     2
21      109089   12     2
38      67497    1      3
                 2      3
                 3      3
                 4      3
                 5      2
                 6      2
                 7      3
                 8      3
                 9      3
                 10     3
                 11     3
                 12     2
                 13     2
                 14     3
                 15     3
                 16     3
                 20     2
                 21     3
                 22     3
                 23     3
                       ..
130401  279722   17     2
                 18     2
                 20     2
                 21     2
                 23     2
                 24     3
                 

In [224]:
train_activity[train_activity['acc_id'] == 20]

,day,acc_id,char_id,server,playtime,npc_kill,solo_exp,party_exp,quest_exp,rich_monster,death,revive,exp_recovery,fishing,private_shop,game_money_change,enchant_count
12324,1,20,149667,ag,0.285560,3.840013,0.028034,0.000000,0.000434,0,0.000000,0.000000,0.0,0.000000,0.000000,-0.041201,0.0
67248,2,20,149667,ag,1.034570,4.073868,0.029362,0.004022,0.001084,0,0.000000,0.000000,0.0,0.000000,0.000000,0.014362,0.0
122231,3,20,149667,ag,0.325351,0.200736,0.001881,0.000000,0.000000,0,0.000000,0.000000,0.0,0.000000,0.000000,0.024104,0.0
177173,4,20,149667,ag,0.550054,3.785943,0.028116,0.000000,0.000434,0,0.000000,0.000000,0.0,0.000000,0.000000,-0.004919,0.0
232358,5,20,149667,ag,1.418437,4.169167,0.034621,0.000461,0.001193,1,0.000000,0.000000,0.0,0.640910,0.000000,-0.004459,0.0
288062,6,20,149667,ag,1.191394,7.581686,0.057354,0.015084,0.000434,0,0.000000,0.000000,0.0,0.000000,0.000000,-0.005596,0.0
342441,7,20,149667,ag,0.720922,3.634546,0.025195,0.005760,0.000434,0,0.000000,0.000000,0.0,0.000000,0.000000,0.020230,0.0
398774,8,20,149667,ag,0.112351,0.001690,0.000100,0.000000,0.000217,0,0.000000,0.000000,0.0,0.000000,0.000000,-0.000474,0.0
455282,9,20,149667,ag,0.655383,0.504545,0.004267,0.000000,0.000868,1,0.000000,0.000000,0.0,0.000000,0.000000,-0.033433,0.0
511195,10,20,149667,ag,1.156284,0.347064,0.005218,0.002828,0.000217,0,0.000000,0.000000,0.0,0.555281,0.000000,0.034422,0.0


In [235]:
# 접속일수
acc_char_attendant = train_activity.groupby(['acc_id', 'char_id', 'day'])['server'].count().unstack().applymap(lambda x : 1. if x >= 1. else 0).astype(np.uint8)
acc_char_attendant.columns = ['day' + str(col) for col in acc_char_attendant.columns]
acc_char_attendant.head(11)

day1  day2  day3  day4  day5  day6  day7  day8  day9  day10  \
acc_id char_id                                                                
2      233598      1     1     1     1     1     1     1     1     1      1   
5      22260       0     0     0     0     0     0     0     0     0      0   
       175813      0     0     0     0     0     0     0     0     0      0   
       321231      0     0     0     0     0     0     0     0     1      0   
8      43437       1     0     1     1     0     1     1     1     1      1   
       71899       1     1     0     0     1     1     1     0     0      0   
       126227      1     1     1     1     1     1     1     1     1      1   
       207949      1     0     0     1     1     0     1     0     1      1   
       257304      1     1     1     1     1     1     1     1     1      1   
       305373      1     1     1     1     1     1     1     0     1      0   
       320423      1     1     1     1     1     1     1     0     1      0   

                ...    day19  day20  day21  day22  day23  day24  day25  day26  \
acc_id char_id  ...                                                             
2      233598   ...        1      1      1      1      1      1      1      1   
5      22260    ...        0      0      0      1      1      1      0      1   
       175813   ...        0      0      0      1      1      1      0      0   
       321231   ...        0      1      0      1      1      1      1      1   
8      43437    ...        0      0      1      1      0      1      0      0   
       71899    ...        0      0      1      1      0      0      0      0   
       126227   ...        1      1      1      1      0      1      1      1   
       207949   ...        1      1      1      1      0      1      1      1   
       257304   ...        1      1      1      1      1      1      1      1   
       305373   ...        0      0      1      1      0      1      0      0   
       320423   ...        0      0      1      1      0      1      0      0   

                day27  day28  
acc_id char_id                
2      233598       1      1  
5      22260        1      0  
       175813       0      0  
       321231       1      1  
8      43437        0      1  
       71899        0      0  
       126227       1      1  
       207949       1      0  
       257304       1      1  
       305373       0      0  
       320423       0      0  

[11 rows x 28 columns]

In [270]:
acc_char_info = pd.concat(
            (acc_char_class,     # combat
             acc_char_playtime,  # activity
             acc_char_pledge,    # pledge
             acc_char_source_t0,    # trade
             acc_char_source_t1,
             acc_char_target_t0,
             acc_char_target_t1,
             acc_char_source2_t0,
             acc_char_source2_t1,
             acc_char_target2_t0,
             acc_char_target2_t1,
             acc_char_attendant  # activity : 출석
            ),   
          axis=1)

In [271]:
acc_char_info = pd.concat((acc_char_info, acc_char_attendant.sum(axis=1)), axis=1) # 총 출석일수 추가

In [272]:
acc_char_info = acc_char_info.rename(columns={0 : 'attend_day'})

In [273]:
acc_char_info['daily_playtime'] = acc_char_info['playtime'] / acc_char_info['attend_day']

In [274]:
acc_char_info.head()

class   playtime  pledge_id  source2other_t0  source2other_t1  \
acc_id char_id                                                                 
2      233598     기사  91.533663        NaN             13.0              NaN   
5      22260      요정   0.046813        5.0              NaN              NaN   
       175813     요정   0.011703        3.0              NaN              NaN   
       321231     요정   6.057616       10.0              NaN              NaN   
8      43437     용기사   0.798163       16.0              NaN              NaN   

                other2target_t0  other2target_t1  source2label_t0  \
acc_id char_id                                                      
2      233598               NaN              NaN             21.0   
5      22260                NaN              NaN              NaN   
       175813               NaN              NaN              NaN   
       321231               NaN              NaN              NaN   
8      43437                NaN              NaN              NaN   

                source2label_t1  label2target_t0       ...        day21  \
acc_id char_id                                         ...                
2      233598               1.0              1.0       ...            1   
5      22260                NaN              NaN       ...            0   
       175813               NaN              NaN       ...            0   
       321231               NaN             10.0       ...            0   
8      43437                NaN              NaN       ...            1   

                day22  day23  day24  day25  day26  day27  day28  attend_day  \
acc_id char_id                                                                
2      233598       1      1      1      1      1      1      1          28   
5      22260        1      1      1      0      1      1      0           5   
       175813       1      1      1      0      0      0      0           3   
       321231       1      1      1      1      1      1      1          10   
8      43437        1      0      1      0      0      0      1          16   

                daily_playtime  
acc_id char_id                  
2      233598         3.269059  
5      22260          0.009363  
       175813         0.003901  
       321231         0.605762  
8      43437          0.049885  

[5 rows x 41 columns]

In [276]:
acc_char_info.shape

(152483, 41)

In [275]:
acc_char_info.isnull().sum()

class                2632
playtime                0
pledge_id           73176
source2other_t0    144179
source2other_t1    120281
other2target_t0    131281
other2target_t1    120715
source2label_t0    143995
source2label_t1    119992
label2target_t0    119893
label2target_t1    120154
day1                    0
day2                    0
day3                    0
day4                    0
day5                    0
day6                    0
day7                    0
day8                    0
day9                    0
day10                   0
day11                   0
day12                   0
day13                   0
day14                   0
day15                   0
day16                   0
day17                   0
day18                   0
day19                   0
day20                   0
day21                   0
day22                   0
day23                   0
day24                   0
day25                   0
day26                   0
day27                   0
day28       

### 위에 추가할 수 있는 변수
- pledge
    1. 혈맹에 들어있는가
    2. ~~몇 위 혈맹에 들어있는가~~
        - 혈맹의 활동과 나의 활동 비교 ; 유저의 적극성 반영
        - 혈맹의 객관적 수치 반영 : 혈맹원 수, 평균 싸움 횟수
        - 혈맹 이동
    3. 들어있는 혈맹의 특성은 어떠한가
- activity
    1. 일별 활동 통계는 어떠한가
    2. 28일 종합 통계는 어떠한가
    3. death와 revive, exp_recovery 관계는 어떠한가
    4. playtime과 fishing, private_shop 운영 시간은 어떠한가
    5. 기존 baseline에서 유용한 변수로 지목받은 game_money_change를 어떻게 살릴 수 있는가
    6. 일일 경험치, 시간변수들로 하루 활동을 어떻게 보내는지 파악
    7. server를 옮겨다니며 플레이하는가
- combat & activity
    1. 활동 시간 및 통계량에 비해 level 증가 추세 (게임의 활력 부여)
    2. 고레벨의 유저와 레벨업하는 유저, 혹은 그 외의 category 분리
    3. 전투 활동을 포함하여 activity의 6.을 같이 파악
- combat & trade
    1. 직업별로 알맞은 무기와 방어구, 악세사리를 사는가
    2. 어떤 직업이 장사꾼으로 많은가
- activity & trade
    1. private_shop 운영과 거래 가운데 어떠한 현상이 발생하는가
    2. enchant_scroll을 구입하고 enchant_count가 증가하는가
        - 위의 행동을 취한 후에 실제로 캐릭터 성적이 올랐는가
    3. game_money_change랑 엮어서 접는 사람을 확실하게 파악해야 한다! - churn 예측

In [277]:
acc_char_info['isin_pledge'] = acc_char_info['pledge_id'].map(lambda x : 0 if np.isnan(x) else 1)

In [ ]:
# 가입한 혈맹의 순위
bbb = tr_pledge.pledge_id.value_counts().to_frame().reset_index()
bbb.columns = ['pledge_id','count']
bbb['rank'] = bbb['count'].rank(ascending = False, method = 'min')
pledge_rank = {}
# dictionary를 활용해 원데이터의 pledge_id에 rank값을 mapping
for i, j in enumerate(list(bbb['rank'])):
    pledge_rank[bbb['pledge_id'][i]] = j
tr_pledge['pledge_rank'] = tr_pledge['pledge_id'].map(pledge_rank)

In [290]:
train_pledge.head()

,day,acc_id,char_id,server,pledge_id,play_char_cnt,combat_char_cnt,pledge_combat_cnt,random_attacker_cnt,random_defender_cnt,same_pledge_cnt,temp_cnt,etc_cnt,combat_play_time,non_combat_play_time
0,1,106660,345122,aa,381,0.036089,0.036089,0.0,0.0,0.0,0.0,0.000000,0.007078,0.027568,0.000000
1,1,46383,161773,aa,10892,0.036089,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.006139
2,1,18495,321373,aa,13705,0.036089,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000347
3,1,118293,366707,aa,2982,0.036089,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.001042
4,1,50421,158063,aa,27104,0.216536,0.072179,0.0,0.0,0.0,0.0,0.098883,0.056626,0.083978,0.000000


In [292]:
test = train_pledge.groupby(['pledge_id', 'day']).agg(
                {'play_char_cnt' : np.mean,
                 'combat_char_cnt' : np.mean,
                 'char_id' : 'count',
                 'pledge_combat_cnt' : np.mean,
                 'random_attacker_cnt' : np.mean,
                 'random_defender_cnt' : np.mean,
                 'same_pledge_cnt' : np.mean,
                 'temp_cnt' : np.mean,
                 'etc_cnt' : np.mean,
                 'combat_play_time' : np.mean})

In [314]:
test

play_char_cnt  combat_char_cnt  char_id  pledge_combat_cnt  \
pledge_id day                                                               
2         1         0.036089         0.000000        1           0.000000   
          23        0.036089         0.000000        1           0.000000   
3         1         0.748252         0.476379       15           0.095192   
          2         0.744987         0.443383       14           0.197998   
          3         0.757876         0.577429       13           0.471708   
          4         0.711475         0.474317       14           0.392743   
          5         0.739831         0.538762       14           1.163530   
          6         0.721786         0.469161       12           0.092885   
          7         0.680145         0.505250       13           0.326287   
          8         0.669040         0.535788       13           0.329509   
          9         0.721786         0.613518       12           0.837783   
          10        0.676073         0.577429       15           0.672898   
          11        0.634571         0.369916       12           0.125364   
          12        0.634571         0.532317       12           0.816383   
          13        1.190948         1.046590       20           1.067263   
          14        1.371394         1.154858       25           1.491618   
          15        1.251578         1.042260       25           1.667990   
          16        1.263126         1.082680       22           0.531810   
          17        1.299215         1.010501       24           0.571878   
          18        1.263126         1.118769       22           1.074548   
          19        1.190948         1.010501       21           2.147275   
          20        1.263126         0.866144       22           1.467942   
          21        1.325588         1.007725       26           0.689980   
          22        1.356126         1.041038       26           2.211790   
          23        1.443573         1.082680       27           1.914153   
          24        1.407483         1.082680       28           1.435159   
          25        1.371394         1.082680       26           1.826732   
          26        1.284780         1.076905       25           3.178622   
          27        1.407483         1.046590       28           1.238462   
          28        1.443573         1.190948       29           0.930668   
...                      ...              ...      ...                ...   
39623     4         0.541340         0.541340       14           0.000000   
          5         0.541340         0.541340       14           0.000000   
          6         0.541340         0.541340       14           0.000000   
          7         0.541340         0.469161       14           0.000000   
          8         0.541340         0.541340       14           0.000000   
          9         0.541340         0.505250       14           0.000000   
          10        0.577429         0.541340       14           0.000000   
          11        0.541340         0.541340       14           0.000000   
          12        0.541340         0.505250       14           0.000000   
          13        0.541340         0.541340       14           0.000000   
          14        0.541340         0.433072       14           0.000000   
          15        0.541340         0.541340       14           0.000000   
          16        0.577429         0.541340       14           0.000000   
          17        0.541340         0.469161       14           0.000000   
          18        0.541340         0.469161       14           0.000000   
          19        0.541340         0.541340       14           0.000000   
          20        0.577429         0.396983       14           0.000000   
          21        0.505250         0.469161       14           0.000000   
          22        0.505250         0.505250       14           0.000000   
          2

In [315]:
test.reset_index()[['pledge_id', 'day']].groupby('pledge_id')['day'].count()

pledge_id
2         2
3        28
4         2
8         9
9         1
12       28
14        2
17       28
18       12
20        6
21       14
22        4
24        1
26        6
27       16
29        4
31        1
37        1
39       28
40        2
41        2
42       28
43       28
44        6
46        3
48       10
50        1
51        1
56       24
57       28
         ..
39579     1
39580     1
39581     1
39582     1
39585    24
39591    28
39592    28
39593    28
39594     6
39595     5
39598     3
39599    10
39600     3
39601    28
39602    28
39603    20
39606     3
39608     7
39609     4
39610     1
39611     1
39613     7
39615    28
39616    13
39618    28
39621    13
39622     6
39623    28
39624     4
39625     1
Name: day, Length: 21860, dtype: int64

In [309]:
acc_char_info['pledge_active_day'] = acc_char_info['pledge_id'].replace(dict(test.reset_index()[['pledge_id', 'day']].groupby('pledge_id')['day'].count())).fillna(0).astype(np.uint8)

In [310]:
acc_char_info

class   playtime  pledge_id  source2other_t0  source2other_t1  \
acc_id char_id                                                                 
2      233598     기사  91.533663        NaN             13.0              NaN   
5      22260      요정   0.046813        5.0              NaN              NaN   
       175813     요정   0.011703        3.0              NaN              NaN   
       321231     요정   6.057616       10.0              NaN              NaN   
8      43437     용기사   0.798163       16.0              NaN              NaN   
       71899      전사   0.142780       10.0              NaN              NaN   
       126227     기사   4.952826       27.0              NaN              4.0   
       207949    마법사   0.837955       20.0              NaN              NaN   
       257304   다크엘프  10.507201       28.0              NaN              1.0   
       305373     요정   0.468131       14.0              NaN              2.0   
       320423    환술사   1.090745        3.0              NaN              NaN   
17     135044     기사  25.213538       28.0              NaN             30.0   
       142415     기사   0.002341        NaN              NaN              NaN   
       176285     기사   0.035110        NaN              NaN              NaN   
       207948     군주   3.213720       18.0              NaN              1.0   
20     91720      전사   2.979654        NaN              1.0              NaN   
       149667   다크엘프  22.608389       36.0              NaN              5.0   
21     109089     군주  21.821929       27.0              NaN              2.0   
       234178     기사   0.009363        NaN              NaN              NaN   
31     142889     전사   0.280879        NaN              NaN              NaN   
       346526     기사  81.801219       28.0              NaN              NaN   
38     23475      전사   2.186172        NaN              NaN              NaN   
       67497      기사  68.623330       21.0              NaN             11.0   
41     240864    환술사   0.004681        1.0              NaN              NaN   
43     168478    용기사   0.021066        NaN              NaN              NaN   
       388134    환술사   4.976233       27.0              NaN              NaN   
50     78995      전사  77.723797        NaN             34.0              NaN   
53     60446    다크엘프  88.893404       46.0              NaN              7.0   
       91157     용기사   0.025747        NaN              NaN              NaN   
       253656     기사   0.002341        NaN              NaN              NaN   
...              ...        ...        ...              ...              ...   
130449 213669    용기사   0.891790       12.0              NaN              NaN   
       220751     전사   0.330032        5.0              NaN              NaN   
       238395    마법사   2.757292       10.0              NaN              NaN   
       240037   다크엘프   0.051494        1.0              NaN              NaN   
       311031     군주   0.507922        9.0              NaN              NaN   
130459 22875      기사  91.011697        NaN             57.0              1.0   
130461 233335     군주   0.301945        3.0              NaN              1.0   
130462 274672     기사   5.762693        NaN              NaN              NaN   
130463 14261     마법사  22.903311       21.0              NaN              NaN   
       292737    마법사  26.751348       24.0              NaN              NaN   
130468 62502      군주   1.380987        3.0              NaN              NaN   
       71078      기사   0.004681        NaN              NaN              NaN   
       179289     기사   0.004681        NaN              NaN              NaN   
       246622     기사   0.004681        NaN              NaN              NaN   
       321120     요정   0.002341        NaN              NaN              NaN   
       397373     기사   0.007022        NaN              NaN              NaN   
130469 6255      용기사   8.119733        NaN              3.0             13.0   
       69296   

,class,playtime,pledge_id,source2other_t0,source2other_t1,other2target_t0,other2target_t1,source2label_t0,source2label_t1,label2target_t0,...,day23,day24,day25,day26,day27,day28,attend_day,daily_playtime,isin_pledge,day
0,요정,0.011703,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,0,0,0,0,3,0.003901,1,28
1,환술사,1.090745,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,0,0,18,0.060597,1,28
2,전사,0.566439,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,1,1,1,0,3,0.188813,1,28
3,군주,0.007022,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,3,0.002341,1,28
4,용기사,0.400252,3,NaN,NaN,NaN,NaN,NaN,1.0,2.0,...,0,0,0,0,0,0,3,0.133417,1,28
5,요정,0.971372,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,8,0.121421,1,28
6,요정,0.313648,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,1,0,0,4,0.078412,1,28
7,전사,6.254231,3,NaN,NaN,NaN,6.0,NaN,NaN,1.0,...,0,0,0,1,1,1,3,2.084744,1,28
8,용기사,0.189593,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,4,0.047398,1,28
9,군주,0.011703,3,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,0,0,0,1,0,0,3,0.003901,1,28
